In [1]:
import os

base_results_dir = "/home/yb107/cvpr2025/DukeDiffSeg/outputs/diffunet-binary-iterative/6.0/final_results"

# subdir structure inference_<organ_name>_inference_test
organ_names = [
    "liver",
    "kidneys",
    "spleen",
    "pancreas",
    "gallbladder",
    "urinary_bladder",
    "duodenum",
    "rectum",
    "colon",
    "stomach",
]

# Create a json file listing all the organ paths for each patient
data = dict()

# list all patients
patients = os.listdir(os.path.join(base_results_dir, "inference_liver_inference_test"))
patients = [p for p in patients if p.endswith("_label.nii.gz")]
# find dirs with _label.nii.gz
for patient in patients:
    organ_paths = dict()
    for organ in organ_names:
        organ_path = os.path.join(
            base_results_dir, f"inference_{organ}_inference_test", patient
        )
        if not os.path.exists(organ_path):
            raise FileNotFoundError(f"File not found: {organ_path}")
        organ_paths[organ] = organ_path
    data[patient] = organ_paths

In [2]:
import nibabel as nib
import numpy as np

ORGAN_NAMES = {
    1: "colon",
    2: "rectum",
    3: "small_bowel",
    4: "stomach",
    5: "liver",
    6: "spleen",
    7: "kidneys",
    9: "pancreas",
    10: "urinary_bladder",
    11: "duodenum",
    12: "gallbladder",
}
organ_name_to_label = {v: k for k, v in ORGAN_NAMES.items()}


def combine_organ_masks(patient_organ_paths, output_path):

    combined_mask = None
    for organ, path in patient_organ_paths.items():
        organ_img = nib.load(path)
        organ_data = organ_img.get_fdata()

        if combined_mask is None:
            combined_mask = np.zeros_like(organ_data)
        combined_mask[organ_data > 0] = organ_name_to_label[
            organ
        ]  # Assuming binary masks

    combined_img = nib.Nifti1Image(combined_mask, organ_img.affine)
    nib.save(combined_img, output_path)

In [19]:
patients = list(data.keys())
combine_organ_masks(
    data[patients[0]],
    f"/home/yb107/cvpr2025/DukeDiffSeg/outputs/diffunet-binary-iterative/6.0/final_results/combined/{patients[0]}",
)

In [4]:
for p in patients:
    output_path = f"/home/yb107/cvpr2025/DukeDiffSeg/outputs/diffunet-binary-iterative/6.0/final_results/combined_label/{p}"
    combine_organ_masks(data[p], output_path)